In [1]:
import pandas as pd
import numpy as np

import dask.dataframe as dd
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

from sklearn.metrics import f1_score, classification_report
from sklearn.feature_selection import SelectFromModel

import pickle

In [2]:
from until import ColumnSelector, undersample_df_by_target, run_grid_search, treshold_search, preprocess_data_train, preprocess_data_test, select_type_cols


In [3]:
TRAIN_DATA = 'data/data_train.csv'
FEATURES_DATA = 'data/features.csv'
RANDOM_STATE = 9

### Подготовка данных

In [4]:
train_df = pd.read_csv(TRAIN_DATA)

In [5]:
train_df['buy_time'] = pd.to_datetime(train_df['buy_time'], unit='s')

In [6]:
X_train = undersample_df_by_target(train_df, 'target')

In [7]:
X_train, true_offers_ids = preprocess_data_train(X_train, FEATURES_DATA)

In [8]:
with open("data/offer_mark.txt", "w") as file:
    print(*true_offers_ids, file=file, sep="\n")

In [9]:
y_train = X_train['target']

In [10]:
X_train = X_train.drop('target', axis = 1)

In [11]:
f_all, f_binary, f_categorical, f_numeric = select_type_cols(X_train)

### Обучение Модели

In [12]:
f_prep_pipeline = make_pipeline(
    ColumnSelector(columns=f_all),
    FeatureUnion(transformer_list=[
        ("numeric_features", make_pipeline(
            ColumnSelector(f_numeric),
            SimpleImputer(strategy="mean"),
            StandardScaler()
        )),
        ("categorical_features", make_pipeline(
            ColumnSelector(f_categorical),
            SimpleImputer(strategy="most_frequent"),
            OneHotEncoder(handle_unknown='ignore')
        )),
        ("boolean_features", make_pipeline(
            ColumnSelector(f_binary),
        ))
    ])
)

In [13]:
lg_fs_pipe = make_pipeline(
    f_prep_pipeline,
    SelectFromModel(LogisticRegression(penalty='l1', random_state=RANDOM_STATE, solver='liblinear'), max_features = 29),
    LogisticRegression(random_state=RANDOM_STATE)
)

In [14]:
lg_fs_pipe.fit(X_train, y_train)

C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columnselector',
                                  ColumnSelector(columns=['147', '45', '220',
                                                          '184', '52', '241',
                                                          '204', '191', '193',
                                                          '137', '6', 'vas_id',
                                                          '71', '192', '83',
                                                          '148', '3', '44',
                                                          '102', '172', '122',
                                                          '144', '131', '247',
                                                          '223', '228', '182',
                                                          '72', '242', '237', ...])),
                                 ('featureunion',
                                  FeatureUnion(transformer_list=[('numeric_features',
                                                                  Pipeline(steps=[(...
                                                                                  ('onehotencoder',
                                                                                   OneHotEncoder(handle_unknown='ignore'))])),
                                                                 ('boolean_features',
                                                                  Pipeline(steps=[('columnselector',
                                                                                   ColumnSelector(columns=['not_first_offer']))]))]))])),
                ('selectfrommodel',
                 SelectFromModel(estimator=LogisticRegression(penalty='l1',
                                                              random_state=9,
                                                              solver='liblinear'),
                                 max_features=29)),
                ('logisticregression', LogisticRegression(random_state=9))])

In [15]:
with open('data/lg_fs_model.pickle', 'wb') as f:
    pickle.dump(lg_fs_pipe, f, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
preds_proba_train = lg_fs_pipe.predict_proba(X_train)

In [17]:
treshold_search(y_train, preds_proba_train)

Лучшая отсечка : 0.3684210526315789, Метрика F1_macro: 0.8884277923102544
              precision    recall  f1-score   support

         0.0       0.91      0.87      0.89     60186
         1.0       0.87      0.91      0.89     60186

    accuracy                           0.89    120372
   macro avg       0.89      0.89      0.89    120372
weighted avg       0.89      0.89      0.89    120372

